In [1]:
import json
import pandas as pd
import os
import re
import glob
import numpy as np
import warnings 

In [2]:
# Display all columns in the pd data frame
pd.set_option('display.max_columns', None)

# Ignore warning displays
warnings.filterwarnings('ignore')

### Stage 1. Read the files, extract the AODV messages and make data frames for each node

In [3]:
# Set default AODV parameters
# Most of these will not be required so can delete them. 
# Keep for now incase I will build routing tables.
NODE_TRAVERSAL_TIME = 10e-3 # 10 milliseconds
NET_DIAMETER = 20
NET_TRAVERSAL_TIME = 2 * NODE_TRAVERSAL_TIME * NET_DIAMETER
PATH_DISCOVERY_TIME = 2 * NET_TRAVERSAL_TIME
ACTIVE_ROUTE_TIMEOUT = 2 * NET_TRAVERSAL_TIME
DELETE_PERIOD = 5 * NET_TRAVERSAL_TIME
RREQ_RETRIES = 3
RREQ_RATELIMIT = 10
RERR_RATELIMIT = 10
HELLO_INTERVAL = 1 # 1000 milliseconds
ALLOWED_HELLO_LOSS = 2
NEXT_HOP_WAIT = 2 * NODE_TRAVERSAL_TIME
BLACKLIST_TIMEOUT = RREQ_RETRIES * NET_TRAVERSAL_TIME # Double check RREQ_RETRIES, there was atypo in the document.

In [4]:
# Get the current working directory
os.getcwd()

'C:\\Users\\alang\\ITS_CP'

In [5]:
print("Ensure that the directory only contains the necessary.json files")
#directory_path = input("Set Working Directory. Include ./: ")
directory_path = 'Datasets\DS1\Test_3' 

Ensure that the directory only contains the necessary.json files


In [6]:
# Verify that the directory path contains the right files only.
print(os.listdir(directory_path))
print()
print(directory_path)

['dataset3_mod_BHN.csv', 'dataset_merged.csv', 'node-37-manet-routing-compare.pcap-37-0.pcap', 'node-38-manet-routing-compare.pcap-38-0.pcap', 'node-39-manet-routing-compare.pcap-39-0.pcap', 'node-40-manet-routing-compare.pcap-40-0.pcap', 'node-41-manet-routing-compare.pcap-41-0.pcap', 'node-42-manet-routing-compare.pcap-42-0.pcap', 'node-43-manet-routing-compare.pcap-43-0.pcap', 'node-44-manet-routing-compare.pcap-44-0.pcap', 'node-45-manet-routing-compare.pcap-45-0.pcap', 'node-46-manet-routing-compare.pcap-46-0.pcap', 'node-47-manet-routing-compare.pcap-47-0.pcap', 'node-48-manet-routing-compare.pcap-48-0.pcap', 'node-49-manet-routing-compare.pcap-49-0.pcap', 'Node_37_Stage_2.csv', 'Node_37_Test3.json', 'Node_38_Stage_2.csv', 'Node_38_Test3.json', 'Node_39_Stage_2.csv', 'Node_39_Test3.json', 'Node_40_Stage_2.csv', 'Node_40_Test3.json', 'Node_41_Stage_2.csv', 'Node_41_Test3.json', 'Node_42_Stage_2.csv', 'Node_42_Test3.json', 'Node_43_Stage_2.csv', 'Node_43_Test3.json', 'Node_44_Stage

In [7]:
# read all the Node files into a list.
# use os.listdir() to get a list of all files in the directory
all_files = os.listdir(directory_path)

# use a list comprehension to filter the .json files
node_files = [file for file in all_files if file.endswith('.json')]

node_files

['Node_37_Test3.json',
 'Node_38_Test3.json',
 'Node_39_Test3.json',
 'Node_40_Test3.json',
 'Node_41_Test3.json',
 'Node_42_Test3.json',
 'Node_43_Test3.json',
 'Node_44_Test3.json',
 'Node_45_Test3.json',
 'Node_46_Test3.json',
 'Node_47_Test3.json',
 'Node_48_Test3.json',
 'Node_49_Test3.json']

**Enter the list of black hole node IDs here**

In [8]:
# This list contains the black hole node IDs.
black_hole_list = ['45', '49']

# Concatenate the ip address to the black hole node ID.
# The ip address may vary per simulation and may need to be modified.
for i, node in enumerate(black_hole_list):
    black_hole_list[i] = '10.1.1.' + node

print(black_hole_list)

['10.1.1.45', '10.1.1.49']


#### Stage 1 functions

In [9]:
# It is assumed that the RREQ is received and not sent.
def get_rreq_info(aodv):
    sn = aodv['aodv.orig_ip']
    dn = aodv['aodv.dest_ip']
    hc = aodv['aodv.hopcount']
    ssn = aodv['aodv.orig_seqno']
    dsn = aodv['aodv.dest_seqno']
    bid = aodv['aodv.rreq_id']
    return sn, dn, hc, ssn, dsn, bid

In [10]:
# It is assumed that the RREP is received and not sent.
def get_rrep_info(aodv):
    sn = aodv['aodv.orig_ip']
    dn = aodv['aodv.dest_ip']
    hc = aodv['aodv.hopcount']
    dsn = aodv['aodv.dest_seqno']
    return sn, dn, hc, dsn

In [11]:
def get_rrer_info(aodv):
    dn = aodv['Unreachable Destinations']['aodv.unreach_dest_ip']
    hc = aodv['aodv.destcount']
    dsn = aodv['Unreachable Destinations']['aodv.dest_seqno']
    return dn, hc, dsn

In [12]:
# Max Sequence Number allowed in AODV is 2**32 - 1 = 4294967295
def calc_seq_inc(seq_rrep, seq_rreq):
    max_seq_num = 2**32 -1
    diff = seq_rrep - seq_rreq
    # If the maximum sequence number has been exceeded, calculate the difference.
    if diff < -1e7:
        diff = max_seq_num + seq_rrep - seq_rreq
    return diff

In [13]:
# Make a list of the node IDs
node_id_list = []
for item in node_files:
    match = re.search(r'\d+', item)
    if match:
        node_id_list.append(match.group())

print(node_id_list)

['37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']


### Stage 1 for loop
#### This will loop through all the .json files and convert them into data frames of AODV messages as seen by each node.

In [14]:
stage_2_list = []

In [15]:
for index in range(len(node_files)):
    # Find the number of the current node and store as a string
    crt_node = '10.1.1.' + node_id_list[index]
    print(f'Current Node: {crt_node}')
    type_4 = 0
    
    # Read the current json file and load it.
    file_path = os.path.join(directory_path, node_files[index])
    j = open(file_path, 'r')
    j1 = json.load(j)
    # Close the JSON file
    j.close()
    
    # Create an empty data frame
    df_aodv = pd.DataFrame(columns = ['Msg_Index', 'frame_time', 'frame_time_relative', 'This_Node', 'Nbr_Node', 
                                      'Next_Hop', 'TTL', 'AODV_Msg', 'Source_Node', 'Destination_Node', 'Hop_Count', 
                                      'Source_Seq_Num', 'Dest_Seq_Num', 'Broadcast_ID'])
    
    # This for loop will iterate through all the AODV messaging lines in the JSON file and convert them into a 
    # data frame that can be further processed in Stage 2.
    for idx in range(0,len(j1)):
        if 'aodv' in j1[idx]['_source']['layers'].keys():
            frame_time = j1[idx]['_source']['layers']['frame']['frame.time_epoch']
            frame_time_rel = j1[idx]['_source']['layers']['frame']['frame.time_relative']
            # Double check the next two final keys as there are multiple options. All appear to have the same values so far.
            ip_src = j1[idx]['_source']['layers']['ip']['ip.src']
            ip_dest = j1[idx]['_source']['layers']['ip']['ip.dst']
            ttl = j1[idx]['_source']['layers']['ip']['ip.ttl']
            # Possibly remove the udp timers.
            udp_time_rel = j1[idx]['_source']['layers']['udp']['Timestamps']['udp.time_relative']
            udp_time_delta = j1[idx]['_source']['layers']['udp']['Timestamps']['udp.time_delta']
            aodv_tp = j1[idx]['_source']['layers']['aodv']['aodv.type']
            if aodv_tp == '1':
                aodv_type = 'RREQ'
                q_j1 = get_rreq_info(j1[idx]['_source']['layers']['aodv'])
                source_node = q_j1[0]
                dest_node = q_j1[1]
                hop_count = q_j1[2]
                s_seq_num = q_j1[3]
                d_seq_num = q_j1[4]
                bc_id = q_j1[5]

            elif aodv_tp == '2':
                aodv_type = 'RREP'
                p_j1 = get_rrep_info(j1[idx]['_source']['layers']['aodv'])
                source_node = p_j1[0]
                dest_node = p_j1[1]
                hop_count = p_j1[2]
                d_seq_num = p_j1[3]
                s_seq_num = None
                bc_id = None

            elif aodv_tp == '3':
                aodv_type = 'RERR'
                r_j1 = get_rrer_info(j1[idx]['_source']['layers']['aodv'])
                source_node = None
                dest_node = r_j1[0]
                hop_count = r_j1[1]
                d_seq_num = r_j1[2]
                s_seq_num = None
                bc_id = None

            elif aodv_tp == '4':
                # This is not relevant and causes issues. It will be removed later along with the messages
                # that respond to it.
                #print('"aodv.type": "4"')
                aodv_type = 'RREP-ACK'
                source_node = None
                dest_node = None
                hop_count = 0
                d_seq_num = 0
                s_seq_num = 0
                bc_id = 0
                type_4 += 1
            
            else:
                print("Error")

            new_row = {'Msg_Index': idx+1, 'frame_time': frame_time, 'frame_time_relative': frame_time_rel,
                       'This_Node': crt_node, 'Nbr_Node': ip_src, 'TTL': ttl, 'Next_Hop': ip_dest, 'AODV_Msg': aodv_type,
                       'Source_Node': source_node, 'Destination_Node': dest_node, 'Hop_Count': hop_count,
                       'Source_Seq_Num': s_seq_num, 'Dest_Seq_Num': d_seq_num, 'Broadcast_ID': bc_id}

            df_aodv = df_aodv.append(new_row, ignore_index=True)
        
    # Convert various columns from string to float or integer.
    df_aodv['Msg_Index'] = df_aodv['Msg_Index'].astype(int)
    df_aodv['frame_time'] = df_aodv['frame_time'].astype(float)
    df_aodv['frame_time_relative'] = df_aodv['frame_time_relative'].astype(float)
    df_aodv['TTL'] = df_aodv['TTL'].astype(int)
    df_aodv['Hop_Count'] = df_aodv['Hop_Count'].astype(int)
    df_aodv['Source_Seq_Num'] = pd.to_numeric(df_aodv['Source_Seq_Num'], errors='coerce').astype('Int64')
    df_aodv['Dest_Seq_Num'] = df_aodv['Dest_Seq_Num'].astype(int)
    df_aodv['Broadcast_ID'] = pd.to_numeric(df_aodv['Broadcast_ID'], errors='coerce').astype('Int64')

    """
    # Write the Stage 1 .csv files
    file_name = f'Node_{node_id_list[index]}_Stage_1.csv'
    dir_path = directory_path + "\\"
    file_path = dir_path + file_name
    df_aodv.to_csv(file_path)
    """

    # Store the data frame in a list for Stage 2
    stage_2_list.append(df_aodv)
    print(f'{type_4} RREP-ACK messages found')
    print()

Current Node: 10.1.1.37
110 RREP-ACK messages found

Current Node: 10.1.1.38
15 RREP-ACK messages found

Current Node: 10.1.1.39
4 RREP-ACK messages found

Current Node: 10.1.1.40
48 RREP-ACK messages found

Current Node: 10.1.1.41
29 RREP-ACK messages found

Current Node: 10.1.1.42
89 RREP-ACK messages found

Current Node: 10.1.1.43
38 RREP-ACK messages found

Current Node: 10.1.1.44
29 RREP-ACK messages found

Current Node: 10.1.1.45
141 RREP-ACK messages found

Current Node: 10.1.1.46
164 RREP-ACK messages found

Current Node: 10.1.1.47
164 RREP-ACK messages found

Current Node: 10.1.1.48
101 RREP-ACK messages found

Current Node: 10.1.1.49
46 RREP-ACK messages found



In [16]:
stage_2_list[0].head(15)

,Msg_Index,frame_time,frame_time_relative,This_Node,Nbr_Node,Next_Hop,TTL,AODV_Msg,Source_Node,Destination_Node,Hop_Count,Source_Seq_Num,Dest_Seq_Num,Broadcast_ID
0,1,0.053304,0.000000,10.1.1.37,10.1.1.46,10.1.1.255,1,RREP,10.1.1.46,10.1.1.46,0,<NA>,0,<NA>
1,2,0.055304,0.002000,10.1.1.37,10.1.1.26,10.1.1.255,1,RREP,10.1.1.26,10.1.1.26,0,<NA>,0,<NA>
2,3,0.071304,0.018000,10.1.1.37,10.1.1.36,10.1.1.255,1,RREP,10.1.1.36,10.1.1.36,0,<NA>,0,<NA>
3,4,0.073050,0.019746,10.1.1.37,10.1.1.38,10.1.1.255,1,RREP,10.1.1.38,10.1.1.38,0,<NA>,0,<NA>
4,5,0.080304,0.027000,10.1.1.37,10.1.1.18,10.1.1.255,1,RREP,10.1.1.18,10.1.1.18,0,<NA>,0,<NA>
5,6,1.068304,1.015000,10.1.1.37,10.1.1.36,10.1.1.255,1,RREP,10.1.1.36,10.1.1.36,0,<NA>,0,<NA>
6,7,1.075050,1.021746,10.1.1.37,10.1.1.38,10.1.1.255,1,RREP,10.1.1.38,10.1.1.38,0,<NA>,0,<NA>
7,8,1.092304,1.039000,10.1.1.37,10.1.1.28,10.1.1.255,1,RREP,10.1.1.28,10.1.1.28,0,<NA>,0,<NA>
8,9,2.020304,1.967000,10.1.1.37,10.1.1.44,10.1.1.255,1,RREP,10.1.1.44,10.1.1.44,0,<NA>,0,<NA>
9,10,2.050304,1.997000,10.1.1.37,10.1.1.26,10.1.1.255,1,RREP,10.1.1.26,10.1.1.26,0,<NA>,0,<NA>


### Stage 2. Extract certain information from the messaging and modify the data frames

**Set certain parameters for creating the table.**

In [17]:
# The increment below is considered to be too high and indicative of a BHN
# This value needs to be determined after unning simulations. 20 is an example
HIGH_DEST_SEQ_NUM_INC = 10
# Assume maximum response time for RREP to be received after and RREQ is sent
# This is currently set to more than enough time but we want to ensure that no matching RREQ messages were missed.
MAX_RESP_TIME = 5

### Stage 2 for loop
#### This will loop through all the data frames created in Stage 1 and add some new features for each row.

In [18]:
stage_3_list = []

In [19]:
for df_idx in range(len(stage_2_list)):
    df_mod = stage_2_list[df_idx].copy()
    # Maybe remove these two features in the future.
    df_mod['This_Node_Is_Dest'] = ""
    df_mod['This_Node_Is_Orig'] = ""
    # Create new columns.
    df_mod['Hello'] = ""
    df_mod['Nbr_Is_Orig'] = ""
    df_mod['Nbr_Is_Dest'] = ""
    df_mod['RREQ_Msg_Idx'] = ""
    df_mod['RREP_Resp_Time'] = ""
    df_mod['RREP_Resp_Time_Per_Hop'] = ""
    df_mod['Hop_Cnt_Over_1'] = ""
    df_mod['Dest_Seq_Num_Increment'] = ""
    df_mod['Orig_Seq_Num_Increment'] = ""
    # This indicates which rows will be deleted in the dataset creation.
    df_mod['Tagged_For_Del'] = ""
    
    type_4 = 0
    no_rreq = 0
    
    # Rows that are affected by the RREP-ACK message will be removed.
    delete_list = []
    print(f'Working on Node {node_id_list[df_idx]} *******************************************')
    
    # Modify the data frame to create more features.
    for idx in range(len(df_mod)):
        # RERR Messages do not contain useful information and cause problems in the pcap file.
        # They will be removed.
        if df_mod['AODV_Msg'][idx] == 'RREP-ACK':
            delete_list.append(df_mod['Msg_Index'][idx])
            type_4 += 1
            df_mod['Tagged_For_Del'][idx] = 'RREP-ACK'
        
        # Check if This Node is source or destination
        if df_mod['This_Node'][idx] == df_mod['Source_Node'][idx]:
            df_mod['This_Node_Is_Orig'][idx] = 'Yes'           
        else:
            df_mod['This_Node_Is_Orig'][idx] = 'No'
            
        if df_mod['This_Node'][idx] == df_mod['Destination_Node'][idx]:
            df_mod['This_Node_Is_Dest'][idx] = 'Yes'
        else:
            df_mod['This_Node_Is_Dest'][idx] = 'No'
         
        # Extract information from the RREQ messages
        if df_mod['AODV_Msg'][idx] == 'RREQ':
            if df_mod['Nbr_Node'][idx] == df_mod['Source_Node'][idx]:
                df_mod['Nbr_Is_Orig'][idx] = True
            else:
                df_mod['Nbr_Is_Orig'][idx] = False

        # Extract information from the RREP messages
        # Determine if the RREP message is a "Hello" broadcast.
        if df_mod['AODV_Msg'][idx] == 'RREP':
            if (df_mod['Next_Hop'][idx].endswith('.255') and 
               (df_mod['Source_Node'][idx] == df_mod['Destination_Node'][idx])):
                # Condition met for Hello, label as such
                df_mod['Hello'][idx] = True

            # If RREP is not a "Hello" broadcast.
            else:
                df_mod['Hello'][idx] = False
                # Determine if the neighbor was the destination node of the RREP
                if ((df_mod['Source_Node'][idx] != df_mod['Destination_Node'][idx]) and
                    (df_mod['Nbr_Node'][idx] == df_mod['Destination_Node'][idx])):
                    # The neighbour is the destination of the RREP message
                    df_mod['Nbr_Is_Dest'][idx] = True

                else:
                    # Neighbor is not the destination node
                    df_mod['Nbr_Is_Dest'][idx] = False

                candidate_idx = None
                idx2 = idx # This starts from the current RREP but will not crash in the first iteration.
                
                # Only RREQ messages within the max response time will be checked.
                # This while loop will determine the 'candidate' or correct RREQ message to which the RREP responded.
                # Modify code and add comments here on how this is done once the corrected .pcap files have been obtained.
                while (df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][idx2]) < MAX_RESP_TIME:
                    #print(idx2, df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][idx2])
                    if df_mod['AODV_Msg'][idx2] == 'RREQ':
                        #print(idx2, df_mod['Msg_Index'][idx2], df_mod['frame_time_relative'][idx2], df_mod['AODV_Msg'][idx2], 
                              #df_mod['Next_Hop'][idx2],
                              #df_mod['Source_Node'][idx2], df_mod['Destination_Node'][idx2])
                        # Determine if RREQ is valid for RREP
                        # It will be valid if the Source_Node and Destination_Node are the same and RREP Hop_Count < RREQ TTL
                        # This code will be modified once the RREP in the .pcap files are correct!!!
                        if ((df_mod['Source_Node'][idx2] == df_mod['Source_Node'][idx]) and 
                            (df_mod['Destination_Node'][idx2] == df_mod['Destination_Node'][idx]) and 
                            (df_mod['TTL'][idx2] >= df_mod['Hop_Count'][idx])):
                            if candidate_idx == None:
                                #print()
                                #print(f"Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]}, RREP Hop_Count: {df_mod['Hop_Count'][idx]}")
                                candidate_idx = idx2
                            else:
                                if df_mod['frame_time_relative'][idx2] <= df_mod['frame_time_relative'][candidate_idx]:
                                    #print()
                                    #print(f"Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]}, 
                                    #RREP Hop_Count: {df_mod['Hop_Count'][idx]}")
                                    candidate_idx = idx2
                                #else:
                                    #print()
                                    #print(f"Not A Candidate, idx2: {idx2}, TTL: {df_mod['TTL'][idx2]},
                                    #RREP Hop_Count: {df_mod['Hop_Count'][id
                    
                    # Decremenet the index counter.
                    idx2 -= 1
                    if idx2 < 0: # If the index is less than 0, break the loop
                        break
    
                # If the correct RREQ message was found pertaining to the RREP
                if candidate_idx != None:
                    #print(f"final candidate: {candidate_idx}, TTL: {df_mod['TTL'][candidate_idx]}")
                    # Update the required columns
                    df_mod['RREQ_Msg_Idx'][candidate_idx] = idx
                    df_mod['RREQ_Msg_Idx'][idx] = candidate_idx
                    df_mod['RREP_Resp_Time'][idx] = \
                        df_mod['frame_time_relative'][idx] - df_mod['frame_time_relative'][candidate_idx]
                    # + 1 added to the demoniator as the hop count should be incrememented when recieved by the node. 
                    # This will also avoid a demoniator of 0.
                    df_mod['RREP_Resp_Time_Per_Hop'][idx] = \
                        df_mod['RREP_Resp_Time'][idx] / (df_mod['Hop_Count'][idx] + 1)
                    df_mod['Dest_Seq_Num_Increment'][idx] = \
                        calc_seq_inc(df_mod['Dest_Seq_Num'][idx], df_mod['Dest_Seq_Num'][candidate_idx])
                    # df_mod['Orig_Seq_Num_Increment'][idx] = calc_seq_inc(df_mod['Orig_Seq_Num'][idx], df_mod['Orig_Seq_Num'][candidate_idx])

                # If the RREQ message was not found
                else:
                    # print(f'No RREQ found for RREP Index {idx}. This is likely due to an error caused by an RREP-ACK message')
                    no_rreq += 1
                    delete_list.append(df_mod['Msg_Index'][idx])
                    df_mod['Tagged_For_Del'][idx] = 'No matching RREQ'

                # Determine if the number of hop counts is greater than 1 if Nbr is not the destination
                if  (df_mod['Nbr_Is_Dest'][idx] == False):
                    if (df_mod['Hop_Count'][idx] > 1):
                        df_mod['Hop_Cnt_Over_1'][idx] = True
                    else:
                        df_mod['Hop_Cnt_Over_1'][idx] = False
                    
    
    # Write the Stage 2 .csv files
    file_name = f'Node_{node_id_list[df_idx]}_Stage_2.csv'
    dir_path = directory_path + "\\"
    file_path = dir_path + file_name
    df_mod.to_csv(file_path)
    
    # The rows will be deleted for Stage 3, but will be in the Stage 2 csv files for reference.
    print(f'{type_4} RREP-ACK messages. and {no_rreq} RREPs with no matching RREQ. They will be deleted')
    # filter the rows to be deleted based on the values in column Msg_Index
    rows_to_delete = df_mod[df_mod['Msg_Index'].isin(delete_list)].index
    # drop the rows from the dataframe
    df_mod = df_mod.drop(index=rows_to_delete)
    # Reset the indices
    df_mod = df_mod.reset_index()

    
    # Store the data frame in a list for Stage 3
    stage_3_list.append(df_mod)
                
    print()

Working on Node 37 *******************************************
110 RREP-ACK messages. and 457 RREPs with no matching RREQ. They will be deleted

Working on Node 38 *******************************************
15 RREP-ACK messages. and 101 RREPs with no matching RREQ. They will be deleted

Working on Node 39 *******************************************
4 RREP-ACK messages. and 6 RREPs with no matching RREQ. They will be deleted

Working on Node 40 *******************************************
48 RREP-ACK messages. and 408 RREPs with no matching RREQ. They will be deleted

Working on Node 41 *******************************************
29 RREP-ACK messages. and 111 RREPs with no matching RREQ. They will be deleted

Working on Node 42 *******************************************
89 RREP-ACK messages. and 781 RREPs with no matching RREQ. They will be deleted

Working on Node 43 *******************************************
38 RREP-ACK messages. and 124 RREPs with no matching RREQ. They will be del

### Stage 3. Create individual Subject Node datasets and merge them into one dataset.

In [20]:
node_dataset_list = []

In [21]:
for df_idx in range (len(stage_3_list)):
    print(f'Working on Node {node_id_list[df_idx]} ***************************************************************')
    df_mod = stage_3_list[df_idx].copy()
    # Remove all rows where the Subject Node is the same as the Neighbour Node
    df_mod = df_mod[df_mod['This_Node'] != df_mod['Nbr_Node']]
    # Reset the indices
    df_mod = df_mod.reset_index()
    # Create an empty dataset
    ds_temp = pd.DataFrame(columns = ['Node', 'Nbr_Node', 'Nbr_Count', 'Hello_Cnt', 'AODV_Msg_Nbr_Cnt', 'RREQs_Sent_To_Nbr',
                                      'RREQs_From_Nbr', 'Nbr_Never_Sends_RREQ', 'Nbr_Is_Orig_Cnt', 'Nbr_Never_Orig', 
                                      'Nbr_Is_Dest_Cnt','Nbr_Never_Dest', 'All_RREPs_Rcvd_This_Node', 'RREPs_From_Nbr', 
                                      'RREP_Resp_Pct', 'Pct_Of_All_RREPs', 'Hop_Cnt_Over_1_Cnt', 'Hop_Cnt_Over_1_Pct', 
                                      'High_Dest_Seq_Num_Inc_Cnt', 'High_Dest_Seq_Num_Inc_Pct', 'Avg_Resp_Dly',
                                      'Avg_Resp_Dly_Per_Hop', 'RERRs_From_Nbr', 'RERRs_From_Nbr_Pct', 'Pct_of_All_Nbrs',
                                      'RREP_To_Nbrs_Ratio', 'Black_Hole_Node'])
    
    # Create Empty lists
    # Concatenation of Source Node and Broadcast ID
    sn_bc_id_list = []
    
    # Set initial counters to 0
    node = df_mod['This_Node'][0]
    All_RREPs_Rcvd_This_Node = 0
    RREQs_Sent_To_Nbr = 0
    # Create the Individual dataset
    for idx in range(0,len(df_mod)):
        # Error in current NS3 files. There should be no messaging between the same node
        if df_mod['Nbr_Node'][idx] != node:
            # If the neighbour node appears for the first time
            if (df_mod['Nbr_Node'][idx] not in ds_temp['Nbr_Node'].values):
                # Create a new row
                new_row = {'Node': node, 'Nbr_Count': 0, 'Nbr_Node': df_mod['Nbr_Node'][idx], 'Hello_Cnt': 0, 
                           'AODV_Msg_Nbr_Cnt': 0, 'RREQs_Sent_To_Nbr': 0, 'RREQs_From_Nbr': 0, 'Nbr_Never_Sends_RREQ': False, 
                           'Nbr_Is_Orig_Cnt': 0, 'Nbr_Never_Orig': False, 'Nbr_Is_Dest_Cnt': 0, 'Nbr_Never_Dest': False, 
                           'All_RREPs_Rcvd_This_Node': 0, 'RREPs_From_Nbr': 0, 'RREP_Resp_Pct': np.nan, 
                           'Pct_Of_All_RREPs': np.nan, 'Hop_Cnt_Over_1_Cnt': 0, 
                           'Hop_Cnt_Over_1_Pct': np.nan, 'High_Dest_Seq_Num_Inc_Cnt': 0, 'High_Dest_Seq_Num_Inc_Pct': np.nan, 
                           'Avg_Resp_Dly': 0, 'Avg_Resp_Dly_Per_Hop': 0, 'RERRs_From_Nbr': 0, 'RERRs_From_Nbr_Pct': np.nan,
                           'Pct_of_All_Nbrs': np.nan, 'RREP_To_Nbrs_Ratio': np.nan, 'Black_Hole_Node': False}

                # Add the new row
                ds_temp = ds_temp.append(new_row, ignore_index=True)

            # Add the appropriate values
            # If message is "Hello Broadcast" then increment counter
            if df_mod['Hello'][idx]:
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Hello_Cnt'] += True

            # check if a message in not a Hello and incremenet the neighbour's counter
            if (df_mod['AODV_Msg'][idx] in ('RREQ', 'RREP', 'RERR')) and (df_mod['Hello'][idx] != True):
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'AODV_Msg_Nbr_Cnt'] += 1

            # Check if TTL > 1
            if ((df_mod['AODV_Msg'][idx] == 'RREQ') and (df_mod['TTL'][idx] > 1) and 
                (df_mod['This_Node'][idx] != df_mod['Nbr_Node'][idx])):
                sn_bc_id = df_mod['Source_Node'][idx] + '_' + str(df_mod['Broadcast_ID'][idx])
                # Increment counter only if the source node has not already sent the RREQ with the same broadcast ID.
                if sn_bc_id not in sn_bc_id_list:
                    sn_bc_id_list.append(sn_bc_id)
                    RREQs_Sent_To_Nbr += 1

            # Set RREQ Features
            if df_mod['AODV_Msg'][idx] == 'RREQ':
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RREQs_From_Nbr'] += 1

                if df_mod['Source_Node'][idx] == df_mod['Nbr_Node'][idx]:
                    ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Nbr_Is_Orig_Cnt'] += 1

                if df_mod['Destination_Node'][idx] == df_mod['Nbr_Node'][idx]:
                    ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Nbr_Is_Dest_Cnt'] += 1

            # Set RREP Features if not a 'Hello' broadcast
            if (df_mod['AODV_Msg'][idx] == 'RREP') and (df_mod['Hello'][idx] == False):
                All_RREPs_Rcvd_This_Node += 1
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RREPs_From_Nbr'] += 1
                # Note: Below the totals delays are being accumulated. They will be averaged by the RREPs_From_Nbr after the loop.
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Avg_Resp_Dly'] += df_mod['RREP_Resp_Time'][idx]
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], \
                        'Avg_Resp_Dly_Per_Hop'] += df_mod['RREP_Resp_Time_Per_Hop'][idx]

                if df_mod['Hop_Cnt_Over_1'][idx]:
                    ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'Hop_Cnt_Over_1_Cnt'] += 1
                if df_mod['Dest_Seq_Num_Increment'][idx] > HIGH_DEST_SEQ_NUM_INC:
                    ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'High_Dest_Seq_Num_Inc_Cnt'] += 1

            # Set RERR Features
            if df_mod['AODV_Msg'][idx] == 'RERR':
                ds_temp.loc[ds_temp['Nbr_Node'] == df_mod['Nbr_Node'][idx], 'RERRs_From_Nbr'] += 1
            
    
    # Add totals to the appropriate columns
    ds_temp['All_RREPs_Rcvd_This_Node'] = All_RREPs_Rcvd_This_Node
    ds_temp['RREQs_Sent_To_Nbr'] = RREQs_Sent_To_Nbr
    
    # Set appropriate data types
    ds_temp['Nbr_Count'] = ds_temp['Nbr_Count'].astype(int)
    ds_temp['Hello_Cnt'] = ds_temp['Hello_Cnt'].astype(int)
    ds_temp['AODV_Msg_Nbr_Cnt'] = ds_temp['AODV_Msg_Nbr_Cnt'].astype(int)
    ds_temp['RREQs_Sent_To_Nbr'] = ds_temp['RREQs_Sent_To_Nbr'].astype(int)
    ds_temp['RREQs_From_Nbr'] = ds_temp['RREQs_From_Nbr'].astype(int)
    ds_temp['Nbr_Never_Sends_RREQ'] = ds_temp['Nbr_Never_Sends_RREQ'].astype(bool)
    ds_temp['Nbr_Is_Orig_Cnt'] = ds_temp['Nbr_Is_Orig_Cnt'].astype(int)
    ds_temp['Nbr_Never_Orig'] = ds_temp['Nbr_Never_Orig'].astype(bool)
    ds_temp['Nbr_Is_Dest_Cnt'] = ds_temp['Nbr_Is_Dest_Cnt'].astype(int)
    ds_temp['Nbr_Never_Dest'] = ds_temp['Nbr_Never_Dest'].astype(bool)
    ds_temp['All_RREPs_Rcvd_This_Node'] = ds_temp['All_RREPs_Rcvd_This_Node'].astype(int)
    ds_temp['RREPs_From_Nbr'] = ds_temp['RREPs_From_Nbr'].astype(int)
    ds_temp['RREP_Resp_Pct'] = ds_temp['RREP_Resp_Pct'].astype(float)
    ds_temp['Pct_Of_All_RREPs'] = ds_temp['Pct_Of_All_RREPs'].astype(float)
    ds_temp['Hop_Cnt_Over_1_Cnt'] = ds_temp['Hop_Cnt_Over_1_Cnt'].astype(int)
    ds_temp['Hop_Cnt_Over_1_Pct'] = ds_temp['Hop_Cnt_Over_1_Pct'].astype(float)
    ds_temp['High_Dest_Seq_Num_Inc_Cnt'] = ds_temp['High_Dest_Seq_Num_Inc_Cnt'].astype(int)
    ds_temp['High_Dest_Seq_Num_Inc_Pct'] = ds_temp['High_Dest_Seq_Num_Inc_Pct'].astype(float)
    ds_temp['Avg_Resp_Dly'] = ds_temp['Avg_Resp_Dly'].astype(float)
    ds_temp['Avg_Resp_Dly_Per_Hop'] = ds_temp['Avg_Resp_Dly_Per_Hop'].astype(float)
    ds_temp['RERRs_From_Nbr'] = ds_temp['RERRs_From_Nbr'].astype(int)
    ds_temp['RERRs_From_Nbr_Pct'] = ds_temp['RERRs_From_Nbr_Pct'].astype(float)
    ds_temp['Pct_of_All_Nbrs'] = ds_temp['Pct_of_All_Nbrs'].astype(float)
    ds_temp['RREP_To_Nbrs_Ratio'] = ds_temp['RREP_To_Nbrs_Ratio'].astype(float)
    ds_temp['Black_Hole_Node'] = ds_temp['Black_Hole_Node'].astype(bool)
    
    # Change the remaining values
    # Change the target variable to True if the Neighbour Node is in the black hole list
    ds_temp.loc[ds_temp['Nbr_Node'].isin(black_hole_list), 'Black_Hole_Node'] = True
    # Number of Neighbors for this Node
    ds_temp['Nbr_Count'] = len(ds_temp['Nbr_Node'].values)
    # Nbr never sends a RREQ
    ds_temp['Nbr_Never_Sends_RREQ'] = (ds_temp['RREQs_From_Nbr'] == 0)
    # Nbr never originates a RREQ
    ds_temp['Nbr_Never_Orig'] = (ds_temp['Nbr_Is_Orig_Cnt'] == 0)
    # Nbr is never the destination of an RREQ
    ds_temp['Nbr_Never_Dest'] = (ds_temp['Nbr_Is_Dest_Cnt'] == 0)

    # Calculate to percentage of RREP Responses from RREQs.
    # There are 2 counters. Decide which one I prefer.
    ds_temp.loc[ds_temp['RREQs_Sent_To_Nbr'] != 0, 'RREP_Resp_Pct'] = \
        round((100 * ds_temp['RREPs_From_Nbr'] / ds_temp['RREQs_Sent_To_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['All_RREPs_Rcvd_This_Node'] != 0, 'Pct_Of_All_RREPs'] = \
        round((100 * ds_temp['RREPs_From_Nbr'] / ds_temp['All_RREPs_Rcvd_This_Node']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'Hop_Cnt_Over_1_Pct'] = \
        round((100 * ds_temp['Hop_Cnt_Over_1_Cnt'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'High_Dest_Seq_Num_Inc_Pct'] = \
        round((100 * ds_temp['High_Dest_Seq_Num_Inc_Cnt'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['RREPs_From_Nbr'] != 0, 'RERRs_From_Nbr_Pct'] = \
        round((100 * ds_temp['RERRs_From_Nbr'] / ds_temp['RREPs_From_Nbr']).astype(float),2)

    ds_temp.loc[ds_temp['Nbr_Count'] != 0, 'Pct_of_All_Nbrs'] = \
        round((100 * 1 / ds_temp['Nbr_Count']).astype(float),2)

    ds_temp.loc[ds_temp['Pct_of_All_Nbrs'] != 0, 'RREP_To_Nbrs_Ratio'] = \
        round((ds_temp['Pct_Of_All_RREPs'] / ds_temp['Pct_of_All_Nbrs']).astype(float),2)

    ds_temp['Avg_Resp_Dly'] = np.where(ds_temp['RREPs_From_Nbr'] != 0, \
           (ds_temp['Avg_Resp_Dly'] / ds_temp['RREPs_From_Nbr']), np.nan)

    ds_temp['Avg_Resp_Dly_Per_Hop'] = np.where(ds_temp['RREPs_From_Nbr'] != 0, \
           (ds_temp['Avg_Resp_Dly_Per_Hop'] / ds_temp['RREPs_From_Nbr']), np.nan)
    
    # Append this data set to the list
    node_dataset_list.append(ds_temp)
    print(f'Completed Working on Node {node_id_list[df_idx]} ***************************************************************')
    print()

Working on Node 37 ***************************************************************
Completed Working on Node 37 ***************************************************************

Working on Node 38 ***************************************************************
Completed Working on Node 38 ***************************************************************

Working on Node 39 ***************************************************************
Completed Working on Node 39 ***************************************************************

Working on Node 40 ***************************************************************
Completed Working on Node 40 ***************************************************************

Working on Node 41 ***************************************************************
Completed Working on Node 41 ***************************************************************

Working on Node 42 ***************************************************************
Completed Working on Node 42 ***

In [22]:
node_dataset_list[0]

,Node,Nbr_Node,Nbr_Count,Hello_Cnt,AODV_Msg_Nbr_Cnt,RREQs_Sent_To_Nbr,RREQs_From_Nbr,Nbr_Never_Sends_RREQ,Nbr_Is_Orig_Cnt,Nbr_Never_Orig,Nbr_Is_Dest_Cnt,Nbr_Never_Dest,All_RREPs_Rcvd_This_Node,RREPs_From_Nbr,RREP_Resp_Pct,Pct_Of_All_RREPs,Hop_Cnt_Over_1_Cnt,Hop_Cnt_Over_1_Pct,High_Dest_Seq_Num_Inc_Cnt,High_Dest_Seq_Num_Inc_Pct,Avg_Resp_Dly,Avg_Resp_Dly_Per_Hop,RERRs_From_Nbr,RERRs_From_Nbr_Pct,Pct_of_All_Nbrs,RREP_To_Nbrs_Ratio,Black_Hole_Node
0,10.1.1.37,10.1.1.46,36,37,100,159,55,False,3,False,0,True,444,28,17.61,6.31,6,21.43,0,0.0,1.470976,0.913592,17,60.71,2.78,2.27,False
1,10.1.1.37,10.1.1.26,36,44,15,159,11,False,0,True,0,True,444,3,1.89,0.68,2,66.67,0,0.0,0.364705,0.172806,1,33.33,2.78,0.24,False
2,10.1.1.37,10.1.1.36,36,21,0,159,0,True,0,True,0,True,444,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.78,0.00,False
3,10.1.1.37,10.1.1.38,36,155,219,159,136,False,0,True,0,True,444,51,32.08,11.49,23,45.10,0,0.0,1.969022,0.543309,32,62.75,2.78,4.13,False
4,10.1.1.37,10.1.1.18,36,48,56,159,35,False,17,False,0,True,444,9,5.66,2.03,6,66.67,0,0.0,1.924230,0.413194,12,133.33,2.78,0.73,False
5,10.1.1.37,10.1.1.28,36,44,0,159,0,True,0,True,0,True,444,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.78,0.00,False
6,10.1.1.37,10.1.1.44,36,17,0,159,0,True,0,True,0,True,444,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.78,0.00,False
7,10.1.1.37,10.1.1.16,36,33,0,159,0,True,0,True,0,True,444,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.78,0.00,False
8,10.1.1.37,10.1.1.25,36,30,94,159,41,False,1,False,0,True,444,16,10.06,3.60,3,18.75,0,0.0,3.229656,1.479032,37,231.25,2.78,1.29,False
9,10.1.1.37,10.1.1.5,36,30,58,159,17,False,2,False,0,True,444,28,17.61,6.31,23,82.14,0,0.0,1.960680,0.523586,13,46.43,2.78,2.27,False


In [23]:
# Create the final dataset
dataset_merged = pd.concat(node_dataset_list)

# Remove the rows where the subject node is a black hole.
# The dataset should be used to train normal nodes to detect black hole nodes.
dataset_merged = dataset_merged[~dataset_merged['Node'].isin(black_hole_list)]

# Reset the indices
dataset_merged = dataset_merged.rename_axis('Index').reset_index()
dataset_merged['Index'] = range(1, len(dataset_merged) + 1)

In [24]:
dataset_merged

,Index,Node,Nbr_Node,Nbr_Count,Hello_Cnt,AODV_Msg_Nbr_Cnt,RREQs_Sent_To_Nbr,RREQs_From_Nbr,Nbr_Never_Sends_RREQ,Nbr_Is_Orig_Cnt,Nbr_Never_Orig,Nbr_Is_Dest_Cnt,Nbr_Never_Dest,All_RREPs_Rcvd_This_Node,RREPs_From_Nbr,RREP_Resp_Pct,Pct_Of_All_RREPs,Hop_Cnt_Over_1_Cnt,Hop_Cnt_Over_1_Pct,High_Dest_Seq_Num_Inc_Cnt,High_Dest_Seq_Num_Inc_Pct,Avg_Resp_Dly,Avg_Resp_Dly_Per_Hop,RERRs_From_Nbr,RERRs_From_Nbr_Pct,Pct_of_All_Nbrs,RREP_To_Nbrs_Ratio,Black_Hole_Node
0,1,10.1.1.37,10.1.1.46,36,37,100,159,55,False,3,False,0,True,444,28,17.61,6.31,6,21.43,0,0.0,1.470976,0.913592,17,60.71,2.78,2.27,False
1,2,10.1.1.37,10.1.1.26,36,44,15,159,11,False,0,True,0,True,444,3,1.89,0.68,2,66.67,0,0.0,0.364705,0.172806,1,33.33,2.78,0.24,False
2,3,10.1.1.37,10.1.1.36,36,21,0,159,0,True,0,True,0,True,444,0,0.00,0.00,0,NaN,0,NaN,NaN,NaN,0,NaN,2.78,0.00,False
3,4,10.1.1.37,10.1.1.38,36,155,219,159,136,False,0,True,0,True,444,51,32.08,11.49,23,45.10,0,0.0,1.969022,0.543309,32,62.75,2.78,4.13,False
4,5,10.1.1.37,10.1.1.18,36,48,56,159,35,False,17,False,0,True,444,9,5.66,2.03,6,66.67,0,0.0,1.924230,0.413194,12,133.33,2.78,0.73,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,10.1.1.48,10.1.1.41,47,10,177,272,97,False,0,True,0,True,949,37,13.60,3.90,32,86.49,0,0.0,3.166573,0.499578,43,116.22,2.13,1.83,False
376,377,10.1.1.48,10.1.1.15,47,25,180,272,128,False,30,False,0,True,949,28,10.29,2.95,15,53.57,0,0.0,1.948167,0.489648,24,85.71,2.13,1.38,False
377,378,10.1.1.48,10.1.1.19,47,2,27,272,19,False,5,False,0,True,949,4,1.47,0.42,4,100.00,0,0.0,0.788844,0.231000,4,100.00,2.13,0.20,False
378,379,10.1.1.48,10.1.1.11,47,5,93,272,44,False,8,False,0,True,949,26,9.56,2.74,25,96.15,0,0.0,3.846910,0.707008,23,88.46,2.13,1.29,False


In [25]:
# Store the final dataset as a .csv file
file_name = f'dataset_merged.csv'
dir_path = directory_path + "\\"
file_path = dir_path + file_name
dataset_merged.to_csv(file_path)